Combines frozen YAMNet + Scaler + Classifier → TensorFlow SavedModel

Supports: LogisticRegression (TF ops), Others → Keras MLP (retrained)

In [31]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import joblib
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


In [32]:
MODELS_DIR = '../models/models_approach1'
RESULTS_DIR = '../results/results_approach1'
SAVED_MODELS_DIR = os.path.join(MODELS_DIR, 'saved_models')
YAMNET_URL = 'https://tfhub.dev/google/yamnet/1'
TARGET_SR = 16000

os.makedirs(SAVED_MODELS_DIR, exist_ok=True)

In [33]:
# LOAD COMPONENTS

label_encoder = joblib.load(os.path.join(MODELS_DIR, 'label_encoder.pkl'))
scaler = joblib.load(os.path.join(MODELS_DIR, 'feature_scaler.pkl'))
comparison_df = pd.read_csv(os.path.join(RESULTS_DIR, 'model_comparison.csv'))

classifiers = {}
for name in comparison_df['Model']:
    path = os.path.join(MODELS_DIR, f'{name.lower()}_model.pkl')
    classifiers[name] = joblib.load(path)

yamnet = hub.load(YAMNET_URL)
print("YAMNet loaded")



YAMNet loaded


In [34]:
#RETRAIN NON-LINEAR CLASSIFIERS AS KERAS MLP

print("\nRETRAINING NON-LR CLASSIFIERS AS KERAS MLP")

X_train = np.load('../data/approach1/train_set/X_train.npy')  
y_train = np.load('../data/approach1/train_set/y_train.npy')
X_train_scaled = scaler.transform(X_train)

keras_classifiers = {}

for name, clf in classifiers.items():
    if isinstance(clf, LogisticRegression):
        print(f"  {name}: Using original (TF-op compatible)")
        keras_classifiers[name] = clf
        continue

    print(f"  {name}: Retraining as Keras MLP...")
    n_classes = len(label_encoder.classes_)

    # Match original architecture
    if 'MLP' in name:
        layers = clf.hidden_layer_sizes
    elif 'Random' in name or 'XGBoost' in name:
        layers = (256, 128)
    elif 'SVM' in name:
        layers = (512,)
    else:
        layers = (256,)

    model = tf.keras.Sequential()
    for i,h in enumerate(layers if isinstance(layers, (list, tuple)) else [layers]):
        model.add(tf.keras.layers.Dense(h, activation='relu'))
        if h > 64: model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(n_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
    model.fit(X_train_scaled, y_train, epochs=50, batch_size=64, verbose=0,
              callbacks=[tf.keras.callbacks.EarlyStopping(
                            monitor='loss',           # ← CHANGE THIS
                            patience=10,
                            restore_best_weights=True
                        )])

    keras_classifiers[name] = model
    print(f"    Retrained")




RETRAINING NON-LR CLASSIFIERS AS KERAS MLP
  XGBoost: Retraining as Keras MLP...
    Retrained
  MLP: Retraining as Keras MLP...
    Retrained
  Random_Forest: Retraining as Keras MLP...
    Retrained
  SVM: Retraining as Keras MLP...
    Retrained
  Logistic_Regression: Using original (TF-op compatible)


In [35]:
# BUILD FULL KERAS MODEL

class YamnetEmbedding(tf.keras.layers.Layer):
    def __init__(self, yamnet_model, **kwargs):
        super().__init__(**kwargs)
        self.yamnet_model = yamnet_model

    def call(self, inputs):
        # inputs shape: (batch, 15360)
        def map_fn(wave):
            scores, embeddings, _ = self.yamnet_model(wave)
            return tf.reduce_mean(embeddings, axis=0)
        return tf.map_fn(map_fn, inputs, dtype=tf.float32)


def build_full_model(classifier, name):
    inputs = tf.keras.Input(shape=(15360,), dtype=tf.float32, name="audio")

    yamnet_layer = YamnetEmbedding(yamnet)
    pooled = yamnet_layer(inputs)

    scaled = (pooled - scaler.mean_) / scaler.scale_

    if isinstance(classifier, LogisticRegression):
        logits = tf.keras.layers.Dense(
            len(label_encoder.classes_),
            kernel_initializer=tf.constant_initializer(classifier.coef_.T),
            bias_initializer=tf.constant_initializer(classifier.intercept_)
        )(scaled)
        probs = tf.nn.softmax(logits)
    else:
        probs = classifier(scaled)

    model = tf.keras.Model(inputs, probs)
    model.compile()
    return model



In [36]:
# BUILD & SAVE ALL

saved_paths = {}
for name, clf in keras_classifiers.items():
    print(f"\nBuilding: {name}")
    full_model = build_full_model(clf, name)

    # Test
    test_audio = np.random.randn(15360).astype(np.float32)
    pred = full_model.predict(test_audio[None, ...], verbose=0)[0]
    print(f"  Probs sum: {pred.sum():.3f}")

    path = os.path.join(SAVED_MODELS_DIR, f"{name.lower()}_full")
    full_model.save(path, save_format='tf')
    saved_paths[name] = path
    print(f"  Saved: {path}")




Building: XGBoost
  Probs sum: 1.000
INFO:tensorflow:Assets written to: ../models/models_approach1\saved_models\xgboost_full\assets


INFO:tensorflow:Assets written to: ../models/models_approach1\saved_models\xgboost_full\assets


  Saved: ../models/models_approach1\saved_models\xgboost_full

Building: MLP
  Probs sum: 1.000
INFO:tensorflow:Assets written to: ../models/models_approach1\saved_models\mlp_full\assets


INFO:tensorflow:Assets written to: ../models/models_approach1\saved_models\mlp_full\assets


  Saved: ../models/models_approach1\saved_models\mlp_full

Building: Random_Forest
  Probs sum: 1.000
INFO:tensorflow:Assets written to: ../models/models_approach1\saved_models\random_forest_full\assets


...

  Saved: ../models/models_approach1\saved_models\random_forest_full

Building: SVM
  Probs sum: 1.000
INFO:tensorflow:Assets written to: ../models/models_approach1\saved_models\svm_full\assets


INFO:tensorflow:Assets written to: ../models/models_approach1\saved_models\svm_full\assets


  Saved: ../models/models_approach1\saved_models\svm_full

Building: Logistic_Regression
  Probs sum: 1.000
INFO:tensorflow:Assets written to: ../models/models_approach1\saved_models\logistic_regression_full\assets


INFO:tensorflow:Assets written to: ../models/models_approach1\saved_models\logistic_regression_full\assets


  Saved: ../models/models_approach1\saved_models\logistic_regression_full


In [37]:
# VERIFY

print("\nVERIFYING SAVEDMODELS")
for name, path in saved_paths.items():
    loaded = tf.keras.models.load_model(path, custom_objects={'KerasLayer': hub.KerasLayer})
    test_out = loaded.predict(np.random.randn(1, 15360).astype(np.float32), verbose=0)[0]
    print(f"  {name}: OK, sum={test_out.sum():.3f}")


inventory = []
for _, row in comparison_df.iterrows():
    name = row['Model']
    size = sum(os.path.getsize(f) for f in tf.io.gfile.glob(f"{saved_paths[name]}/*")) / 1e6
    inventory.append({
        'Model': name,
        'Val_F1': row['Val F1'],
        'Path': saved_paths[name],
        'Size_MB': f"{size:.1f}"
    })

inv_df = pd.DataFrame(inventory).sort_values('Val_F1', ascending=False)
print("\nINVENTORY:")
print(inv_df.to_string(index=False))
inv_df.to_csv(os.path.join(RESULTS_DIR, 'full_model_inventory.csv'), index=False)


VERIFYING SAVEDMODELS
  XGBoost: OK, sum=1.000
  MLP: OK, sum=1.000
  Random_Forest: OK, sum=1.000
  SVM: OK, sum=1.000
  Logistic_Regression: OK, sum=1.000

INVENTORY:
              Model  Val_F1                                                             Path Size_MB
            XGBoost  0.9257             ../models/models_approach1\saved_models\xgboost_full     3.5
                MLP  0.9250                 ../models/models_approach1\saved_models\mlp_full     3.5
      Random_Forest  0.9213       ../models/models_approach1\saved_models\random_forest_full     3.5
                SVM  0.9209                 ../models/models_approach1\saved_models\svm_full     3.5
Logistic_Regression  0.9191 ../models/models_approach1\saved_models\logistic_regression_full     3.6
